In [12]:
# Importing required pakages
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import pandas as pd

In [2]:
# Defing the file path
file_path = r'data\sample.pdf'

In [3]:
# Function to convert pdf to image
def pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)

In [4]:
# Function to extract the text along with text co-ordinates
def extract_text_with_coordinates(image: Image) -> list:
    # Use Tesseract to do OCR on the image
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    parsed_results = []
    for i in range(len(data["text"])):
        # Filter out OCR results with no text
        if data["text"][i].strip() != "":
            parsed_data = {
                "text": data["text"][i],
                "left": data["left"][i],
                "top": data["top"][i],
                "width": data["width"][i],
                "height": data["height"][i],
            }
            parsed_results.append(parsed_data)

    return parsed_results

In [5]:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Users\pis05408.PINNACLE\Desktop\Suraj\pytesseract\tesseract.exe'

In [6]:
# Calling the function to convert PDF to image
images = pdf_to_images(file_path)

In [7]:
all_results = []
for img in images:
    results = extract_text_with_coordinates(img)
    all_results.extend(results)

print(all_results)

# results = extract_text_with_coordinates(image)

# print(results)

[{'text': 'GENERAL', 'left': 7171, 'top': 202, 'width': 310, 'height': 66}, {'text': 'NOTES:', 'left': 7525, 'top': 202, 'width': 264, 'height': 66}, {'text': 'KEY', 'left': 4156, 'top': 275, 'width': 138, 'height': 50}, {'text': 'NOTES:', 'left': 4318, 'top': 274, 'width': 250, 'height': 52}, {'text': '1.', 'left': 7077, 'top': 280, 'width': 45, 'height': 50}, {'text': 'ALL', 'left': 7170, 'top': 280, 'width': 121, 'height': 50}, {'text': 'ELEVATIONS', 'left': 7318, 'top': 279, 'width': 427, 'height': 52}, {'text': 'ARE', 'left': 7768, 'top': 280, 'width': 139, 'height': 50}, {'text': 'PULLED', 'left': 7935, 'top': 280, 'width': 261, 'height': 51}, {'text': 'FROM', 'left': 8225, 'top': 279, 'width': 193, 'height': 52}, {'text': 'FINISH', 'left': 8448, 'top': 279, 'width': 217, 'height': 52}, {'text': 'FLOOR.', 'left': 8695, 'top': 279, 'width': 245, 'height': 52}, {'text': '1]', 'left': 4070, 'top': 356, 'width': 19, 'height': 50}, {'text': 'BALL', 'left': 4156, 'top': 352, 'width': 1

In [11]:
data = pd.DataFrame(columns=['text', 'left', 'top', 'width', 'height'])
data.head()

,text,left,top,width,height


In [14]:
dfs = [data]  # Start with the initial data
for item in all_results:
    dfs.append(pd.DataFrame([item]))
data = pd.concat(dfs, ignore_index=True)

In [17]:
data.head()

,text,left,top,width,height
0,GENERAL,7171,202,310,66
1,NOTES:,7525,202,264,66
2,KEY,4156,275,138,50
3,NOTES:,4318,274,250,52
4,1.,7077,280,45,50


In [58]:
data.to_csv('data/sample_data.csv', index=False)

In [59]:
# ?data.to_csv

### Another Approach

In [18]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image

In [19]:
# Convert PDF to list of images
pages = convert_from_path('data/sample.pdf')

In [27]:
type(pages[0])

PIL.PpmImagePlugin.PpmImageFile

In [87]:
dfs1 = []
for page_number, page in enumerate(pages):
    # Use pytesseract to extract bounding boxes
    boxes = pytesseract.image_to_boxes(page)
    
    for page_number, page in enumerate(pages):
        # Convert the image to RGB mode
        page = page.convert("RGB")

        # Use pytesseract to extract bounding boxes
        boxes = pytesseract.image_to_boxes(page)

        for box in boxes.splitlines():
            char, x, y, w, h, _ = box.split()
            x, y, w, h = int(x), int(y), int(w), int(h)

            # Extract corresponding text
            try:
                cropped_img = page.crop((x, page.height - h, w, page.height - y))
                text = pytesseract.image_to_string(cropped_img).strip()

                list_dict = {'page': (page_number+1), 'text': text, 'x': x, 'y': y, 'w': w, 'h': h}
                dfs1.append(pd.DataFrame([list_dict]))
                # print(f"Page: {page_number + 1}, Text: '{text}', Coordinates: ({x}, {y}, {w}, {h})")
            except Exception as e:
                print(f"Error occurred on page {page_number} with box coordinates {(x, y, w, h)}")
                print(str(e))
                # break
        

Error occurred on page 0 with box coordinates (6044, 0, 6101, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (6119, 0, 6122, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (6041, 0, 6043, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (6064, 0, 6119, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (91, 0, 104, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (106, 0, 117, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (112, 0, 122, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (118, 0, 128, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (129, 0, 138, 0)
tile cannot extend outside image
Error occurred on page 0 with box coordinates (141, 0, 150, 0)
tile cannot extend outside image
Error occurred on page 0 with box

In [76]:
# dfs1

In [88]:
df = pd.DataFrame(pd.concat(dfs1, ignore_index=True))

In [89]:
df.head()

,page,text,x,y,w,h
0,1,G,7171,6932,7230,6998
1,1,,7229,6932,7264,6997
2,1,,7264,6932,7294,6997
3,1,,7294,6932,7355,6997
4,1,,7355,6932,7386,6997


In [92]:
# df.to_csv('data/sample_data_1.csv', index=False)